In [1]:
#C2a
def sendPartition(iter):
    client = MongoClient()
    db = client.TaskC_db
    climate = db.ClimateData
    fire = db.FireData
        
    for line in iter:
        datatype = line[0]
        if datatype == "cdata":
            Climate={}
            Climate["Station"] = int(line[1])
            Climate["Date"] = line[9]
            Climate["Air Temperature(Celcius)"] = float(line[2])
            Climate["Relative Humidity"] = float(line[3])
            Climate["WindSpeed  (knots)"] = float(line[4])
            Climate["Max Wind Speed"] = float(line[5])
            Climate["MAX"] = line[6]
            Climate["MIN"] = line[7]
            Climate["Precipitation"] = line[8]
            Climate["fire"] = []
            climate.insert_one(Climate)
                
        if datatype == "fdata":
            Fire={}
            Fire["Latitude"] = float(line[1])
            Fire["Longitude"] = float(line[2])
            Fire["Surface Temperature (kelvin)"] = float(line[3])
            Fire["Datetime"] = line[8]
            Fire["Power"] = float(line[4])
            Fire["Confidence"] = float(line[5])
            Fire["Date"] = line[7]
            Fire["Surface Temperature (Celcius)"] = float(line[6])
            fire.insert_one(Fire)
            
            climate.update_one({"Date":Fire["Date"]},
                               {"$push":{ "fire": {"_id": Fire["_id"]}}})           

In [ ]:
import pymongo
from pymongo import MongoClient
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

    
sc = SparkContext.getOrCreate()

if (sc is None):
    sc = SparkContext("local[2]","FireClimate")
ssc = StreamingContext(sc, 5)

host = "localhost"
port = 9999

lines = ssc.socketTextStream(host, int(port))
Line = lines.map(lambda line: line.split(","))                          
Line.pprint()
Line.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))

ssc.start()
try:
    ssc.awaitTermination(timeout=800)
except KeyboardInterrupt:
    ssc.stop()
    sc.stop()

ssc.stop()
sc.stop()

-------------------------------------------
Time: 2018-05-21 16:14:40
-------------------------------------------
['cdata', '948700', '19', '56.8', '7.9', '11.1', '72.0*', '61.9*', '0.00I', '2018-05-21 16:14:36']
['fdata', '-37.966', '145.051', '341.8', '26.7', '78', '68', '2018-05-21 16:14:36', '2018-05-21 16:14:36.765382']
['fdata', '-35.541', '143.311', '336.3', '62', '82', '63', '2018-05-21 16:14:36', '2018-05-21 16:14:36.765496']
['fdata', '-35.554', '143.307', '326.8', '23.8', '67', '53', '2018-05-21 16:14:36', '2018-05-21 16:14:36.765704']
['fdata', '-35.543', '143.316', '340.4', '84.2', '86', '67', '2018-05-21 16:14:36', '2018-05-21 16:14:36.765911']
['fdata', '-37.708', '145.1', '327.8', '16.2', '80', '54', '2018-05-21 16:14:36', '2018-05-21 16:14:36.765991']
['cdata', '948700', '15', '50.7', '9.2', '13', '', '58.3', '0.02G', '2018-05-21 16:14:37']
['fdata', '-35.646', '142.282', '305.6', '11.8', '65', '32', '2018-05-21 16:14:37', '2018-05-21 16:14:37.771539']
['fdata', '-38.0

-------------------------------------------
Time: 2018-05-21 16:15:15
-------------------------------------------
['cdata', '948700', '26', '53.7', '8.7', '15', '97.0*', '54.7', '0.00I', '2018-05-21 16:15:10']
['fdata', '-37.236', '141.176', '310.2', '35.1', '68', '37', '2018-05-21 16:15:10', '2018-05-21 16:15:10.866387']
['fdata', '-37.227', '141.151', '308.4', '28.1', '59', '35', '2018-05-21 16:15:10', '2018-05-21 16:15:10.866529']
['fdata', '-37.242', '141.153', '316.9', '56.9', '77', '43', '2018-05-21 16:15:10', '2018-05-21 16:15:10.866592']
['fdata', '-37.238', '141.145', '316.3', '55.6', '76', '43', '2018-05-21 16:15:10', '2018-05-21 16:15:10.866689']
['fdata', '-37.365', '148.052', '337.1', '37.7', '90', '63', '2018-05-21 16:15:10', '2018-05-21 16:15:10.866822']
['cdata', '948700', '21', '64.5', '10.6', '15.9', '86.4*', '59.4*', '1.26G', '2018-05-21 16:15:11']
['fdata', '-37.376', '148.042', '321.2', '17.3', '81', '48', '2018-05-21 16:15:11', '2018-05-21 16:15:11.871841']
['fdat

-------------------------------------------
Time: 2018-05-21 16:15:50
-------------------------------------------
['cdata', '948710', '20', '59.7', '8.6', '15.9', '78.6*', '61.9', '0.01G', '2018-05-21 16:15:45']
['fdata', '-36.1858', '143.885', '312.6', '18.3', '55', '39', '2018-05-21 16:15:45', '2018-05-21 16:15:45.980417']
['fdata', '-36.0966', '142.3635', '334.3', '43.5', '86', '61', '2018-05-21 16:15:45', '2018-05-21 16:15:45.980494']
['fdata', '-36.2111', '141.505', '317.3', '22.1', '68', '44', '2018-05-21 16:15:45', '2018-05-21 16:15:45.980595']
['fdata', '-36.5794', '142.5959', '312.2', '17.1', '53', '39', '2018-05-21 16:15:45', '2018-05-21 16:15:45.980689']
['fdata', '-36.1002', '142.3405', '351.7', '77.1', '96', '78', '2018-05-21 16:15:45', '2018-05-21 16:15:45.980779']
['cdata', '948710', '21', '58.1', '5.3', '11.1', '78.6*', '60.3*', '0.00G', '2018-05-21 16:15:46']
['fdata', '-36.3096', '142.7722', '333.1', '42.8', '85', '59', '2018-05-21 16:15:46', '2018-05-21 16:15:46.9819

-------------------------------------------
Time: 2018-05-21 16:16:25
-------------------------------------------
['cdata', '948720', '13', '48.5', '5.3', '11.1', '64.4*', '51.3*', '0.08G', '2018-05-21 16:16:20']
['fdata', '-37.2794', '143.5903', '317.3', '22.2', '51', '44', '2018-05-21 16:16:20', '2018-05-21 16:16:20.092671']
['fdata', '-37.2777', '143.6017', '329.9', '38.7', '83', '56', '2018-05-21 16:16:20', '2018-05-21 16:16:20.092864']
['fdata', '-37.1402', '144.6153', '334.6', '47.7', '86', '61', '2018-05-21 16:16:20', '2018-05-21 16:16:20.093316']
['fdata', '-37.2811', '143.8819', '316.6', '19.8', '66', '43', '2018-05-21 16:16:20', '2018-05-21 16:16:20.093497']
['fdata', '-38.0372', '146.0616', '340.9', '58.2', '90', '67', '2018-05-21 16:16:20', '2018-05-21 16:16:20.093894']
['cdata', '948720', '12', '44', '4.7', '7', '61.0*', '48.6*', '0.00G', '2018-05-21 16:16:21']
['fdata', '-38.049', '146.0513', '347.3', '71', '94', '74', '2018-05-21 16:16:21', '2018-05-21 16:16:21.098245']


-------------------------------------------
Time: 2018-05-21 16:17:00
-------------------------------------------
['cdata', '948720', '12', '51.6', '3.4', '11.1', '62.8*', '46.2*', '0.05G', '2018-05-21 16:16:55']
['fdata', '-36.7185', '144.5427', '328.4', '34.3', '81', '55', '2018-05-21 16:16:55', '2018-05-21 16:16:55.172804']
['fdata', '-36.754', '141.7932', '315.8', '16.9', '64', '42', '2018-05-21 16:16:55', '2018-05-21 16:16:55.172895']
['fdata', '-36.7212', '144.5501', '333.6', '41.2', '86', '60', '2018-05-21 16:16:55', '2018-05-21 16:16:55.172983']
['fdata', '-36.9315', '142.9519', '326.5', '29.6', '80', '53', '2018-05-21 16:16:55', '2018-05-21 16:16:55.173070']
['fdata', '-36.9054', '143.9176', '328.4', '32.6', '81', '55', '2018-05-21 16:16:55', '2018-05-21 16:16:55.173173']
['cdata', '948720', '14', '50.1', '8.7', '15', '66.6*', '44.2', '0.00G', '2018-05-21 16:16:56']
['fdata', '-36.7988', '141.3436', '319.2', '21.9', '71', '46', '2018-05-21 16:16:56', '2018-05-21 16:16:56.17440

...

-------------------------------------------
Time: 2018-05-21 16:21:05
-------------------------------------------
['fdata', '-37.8169', '142.5723', '321.7', '21.7', '74', '48', '2018-05-21 16:20:59', '2018-05-21 16:20:59.841068']
['fdata', '-37.5837', '144.1565', '313.6', '14.1', '59', '40', '2018-05-21 16:20:59', '2018-05-21 16:20:59.841343']
['fdata', '-37.4611', '144.9371', '342.8', '54.4', '91', '69', '2018-05-21 16:20:59', '2018-05-21 16:20:59.841416']
['fdata', '-37.9158', '141.9147', '314.4', '12.3', '61', '41', '2018-05-21 16:20:59', '2018-05-21 16:20:59.841478']
['fdata', '-37.9143', '141.9264', '314.4', '12.4', '61', '41', '2018-05-21 16:20:59', '2018-05-21 16:20:59.841554']
['fdata', '-37.7862', '142.9823', '317.1', '15', '67', '43', '2018-05-21 16:21:00', '2018-05-21 16:21:00.844445']
['fdata', '-37.7845', '142.9936', '334.1', '37.8', '86', '60', '2018-05-21 16:21:00', '2018-05-21 16:21:00.844902']
['fdata', '-37.7416', '143.2885', '314.3', '11', '61', '41', '2018-05-21 16:

-------------------------------------------
Time: 2018-05-21 16:21:40
-------------------------------------------
['fdata', '-35.1145', '141.2627', '323', '17.1', '76', '49', '2018-05-21 16:21:34', '2018-05-21 16:21:34.957509']
['fdata', '-36.3579', '146.1268', '315.3', '13.5', '63', '42', '2018-05-21 16:21:34', '2018-05-21 16:21:34.957725']
['fdata', '-35.1164', '141.2508', '325.3', '20.2', '78', '52', '2018-05-21 16:21:34', '2018-05-21 16:21:34.957808']
['fdata', '-36.0787', '146.5728', '314.1', '10.8', '60', '40', '2018-05-21 16:21:34', '2018-05-21 16:21:34.957880']
['fdata', '-37.9796', '146.8024', '314.8', '17.3', '62', '41', '2018-05-21 16:21:34', '2018-05-21 16:21:34.957963']
['fdata', '-37.34', '149.3668', '347.6', '61.2', '67', '74', '2018-05-21 16:21:35', '2018-05-21 16:21:35.958592']
['fdata', '-37.8567', '147.1576', '321.4', '19.9', '62', '48', '2018-05-21 16:21:35', '2018-05-21 16:21:35.958819']
['fdata', '-36.7287', '142.6017', '359.1', '82.1', '99', '85', '2018-05-21 16:

-------------------------------------------
Time: 2018-05-21 16:22:15
-------------------------------------------
['fdata', '-36.4324', '141.5582', '325.5', '17.7', '77', '52', '2018-05-21 16:22:10', '2018-05-21 16:22:10.040894']
['fdata', '-38.0052', '142.6179', '362.5', '95.2', '94', '89', '2018-05-21 16:22:10', '2018-05-21 16:22:10.041098']
['fdata', '-38.3557', '147.0372', '325.3', '28.6', '78', '52', '2018-05-21 16:22:10', '2018-05-21 16:22:10.041205']
['fdata', '-35.4509', '143.1522', '320.7', '15.1', '73', '47', '2018-05-21 16:22:10', '2018-05-21 16:22:10.041286']
['fdata', '-38.0962', '146.9724', '320.4', '19.2', '73', '47', '2018-05-21 16:22:10', '2018-05-21 16:22:10.041546']
['fdata', '-35.4625', '143.1486', '317.2', '11.3', '67', '44', '2018-05-21 16:22:11', '2018-05-21 16:22:11.041879']
['fdata', '-38.0942', '146.9603', '313.1', '11.3', '52', '39', '2018-05-21 16:22:11', '2018-05-21 16:22:11.042064']
['fdata', '-38.3499', '147.033', '308.5', '19', '61', '35', '2018-05-21 16

-------------------------------------------
Time: 2018-05-21 16:22:50
-------------------------------------------
['fdata', '-37.7216', '143.4272', '332.8', '33', '85', '59', '2018-05-21 16:22:45', '2018-05-21 16:22:45.125731']
['fdata', '-37.8717', '143.3652', '322.5', '19.2', '75', '49', '2018-05-21 16:22:45', '2018-05-21 16:22:45.125943']
['fdata', '-37.8717', '143.2566', '316.5', '13', '66', '43', '2018-05-21 16:22:45', '2018-05-21 16:22:45.126058']
['fdata', '-37.3851', '145.8878', '336.5', '46.2', '83', '63', '2018-05-21 16:22:45', '2018-05-21 16:22:45.126152']
['fdata', '-37.7446', '143.4085', '339.4', '42.8', '89', '66', '2018-05-21 16:22:45', '2018-05-21 16:22:45.126239']
['fdata', '-36.7159', '141.3991', '322.5', '17.7', '75', '49', '2018-05-21 16:22:46', '2018-05-21 16:22:46.129051']
['fdata', '-35.7626', '142.1167', '336', '33.2', '87', '62', '2018-05-21 16:22:46', '2018-05-21 16:22:46.129370']
['fdata', '-37.8011', '144.7343', '317.2', '12.4', '67', '44', '2018-05-21 16:22

-------------------------------------------
Time: 2018-05-21 16:23:25
-------------------------------------------
['fdata', '-37.3949', '147.0175', '364.6', '96.6', '100', '91', '2018-05-21 16:23:20', '2018-05-21 16:23:20.237326']
['fdata', '-37.671', '144.6536', '331.7', '17.6', '84', '58', '2018-05-21 16:23:20', '2018-05-21 16:23:20.237486']
['fdata', '-37.9277', '143.477', '328.6', '13.1', '66', '55', '2018-05-21 16:23:20', '2018-05-21 16:23:20.237561']
['fdata', '-37.4224', '147.0511', '386.4', '176.6', '100', '113', '2018-05-21 16:23:20', '2018-05-21 16:23:20.237615']
['fdata', '-37.6625', '144.287', '343.7', '42.3', '92', '70', '2018-05-21 16:23:20', '2018-05-21 16:23:20.237669']
['fdata', '-37.4375', '147.0336', '379.2', '152.1', '100', '106', '2018-05-21 16:23:21', '2018-05-21 16:23:21.240810']
['fdata', '-37.4414', '147.0139', '334', '33.9', '73', '60', '2018-05-21 16:23:21', '2018-05-21 16:23:21.241037']
['fdata', '-37.4295', '147.0428', '332.6', '34', '77', '59', '2018-05-21